In [6]:
# librerías de web scrapping y conexión a APIs
import requests


# DS y DA
import pandas as pd
import numpy as np


# Extras y de apoyo
import re
import json
import io
import zipfile
import os
from datetime import datetime


# al diablo, odio las warnings
import warnings
warnings.filterwarnings('ignore')

In [21]:
def nextDate(year : int, month : int) -> tuple[int, int]:
    nextMonth = month + 1 if month < 12 else 1
    nextYear = year if month < 12 else year + 1
    return nextMonth, nextYear

def previousDate(year : int, month : int) -> tuple[int,int]:
    prevMonth = month - 1 if month > 1 else 12
    prevYear = year if month > 1 else year - 1
    return prevMonth, prevYear

In [26]:
def api_request_gtc(response : requests.models.Response) -> tuple[dict, int]:
    if response.status_code == 200:
        zip_file = zipfile.ZipFile(io.BytesIO(response.content))

        for file_name in zip_file.namelist():
            if file_name.endswith('.json'):
                with zip_file.open(file_name) as json_file:
                    datos = json.load(json_file)
        return datos, response.status_code
    else:
        return {}, response.status_code

In [24]:
# condiciones iniciales
year = 2020
month = 1

# condiciones finales
today = datetime.now()
last_month, last_year = previousDate(today.year, today.month)

In [18]:
# sabiendo que solo hay datos a partir de 2020-01-01
# extraemos todos los datos desde dicha fecha
url = "https://ocds.guatecompras.gt/"
endpoint = f"file/json/{year}/{month}"

In [ ]:
while (month <= last_month) and (year <= last_year):
    # solicitamos la información
    try: 
        response = requests.get(url + endpoint)

        data, status = api_request_gtc(response)
        print(status)

        if data:
            with open('../data/data_all_months/' + f'file_{year}-{month}.json', 'w', encoding='UTF-8') as js:
                json.dump(data, js, indent=4)
    except Exception as e:
        print(f'Se obtuvo el siguiente error: \n\{e}')
        with open('../logs/failed_connection.log', 'a') as fail:
            fail.write(e + '\n\n\n')
        break